In [137]:
import numpy as np
from sklearn.linear_model import LinearRegression
import csv
import pandas as pd
import re
import sklearn
from sklearn import preprocessing
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OneHotEncoder

In [138]:
orig = pd.read_csv('data/final_data.csv').drop(['Unnamed: 0'], axis=1) # Remove unncessary column 
orig = orig[orig['year'] != 2013] # 2013 has incomplete data, so we will remove

In [139]:
grouped = pd.DataFrame(orig.groupby(['year', 'month', 'state']).count()['day']).rename(columns={"day":"n_incidents"})
grouped.reset_index(inplace=True)
df = orig.merge(grouped, left_on=['year', 'month', 'state'], right_on=['year', 'month', 'state'])

In [140]:
df = df.drop(['incident_id', 'city_or_county', 'address', 'congressional_district', 'incident_characteristics',
             'participant_age', 'participant_gender', 'participant_name', 'participant_relationship', 
             'participant_status', 'participant_type', 'state_house_district', 'state_senate_district', 
             'gun_stolen', 'gun_type',
             'Permit Type' #FIX THIS LATER
             ], axis=1)

In [141]:
# Need to clean the data (research to isolate which labels are actually synonymous then one hot encode)
# df['Permit Type'].value_counts()

In [142]:
# One-hot encoding states
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(pd.DataFrame(orig['state']))
encoded = pd.DataFrame(enc.transform(orig[['state']]).toarray())

# Assigning states to column names
state_list = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

encoded.columns = state_list

# Merge with original dataframe
df = df.merge(encoded, left_index=True, right_index=True)
df = df.drop(['state'], axis=1)
df.head()

,year,month,day,n_killed,n_injured,latitude,longitude,n_guns_involved,n_people,n_young,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2014,1,5,2,1,39.9740,-75.2433,NaN,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,1,5,1,3,42.1306,-80.0752,NaN,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,1,12,1,1,39.8314,-75.3923,NaN,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,1,15,1,1,40.6139,-75.4453,NaN,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,1,23,1,1,40.4640,-79.9504,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
df['n_guns_involved'] = df['n_guns_involved'].fillna(1)
df = df.fillna(0)

In [148]:
df.columns

Index(['year', 'month', 'day', 'n_killed', 'n_injured', 'latitude',
       'longitude', 'n_guns_involved', 'n_people', 'n_young', 'n_mid', 'n_old',
       'n_female', 'n_male', 'n_unknown_gender', 'n_unknown_age',
       'Mental_Health_Records_Submitted_2008',
       'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008',
       'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score',
       'Gun Deaths per 100k People (2018)',
       'Gun Death Rate (Ranked High to Low)', 'Rank', '# of guns per capita',
       '# of guns registered', 'Handgun_Carry_Permit_Fee', 'Years_Valid',
       '5_Year_Cost', 'Happiness Score', 'time number normed', 'n_incidents',
       'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Mo

In [150]:
# Move n_incidents to the front
n_incidents = df['n_incidents']
df.drop(labels=['n_incidents'], axis=1,inplace = True)
df.insert(0, 'n_incidents', n_incidents)

In [157]:
# MACHINE LEARNING WHOAAAAAAAAAAAAAAAAAAAA
X = df.iloc[:, df.columns != 'n_incidents']
y = df.iloc[:, 0] 

reg = LinearRegression().fit(X,y)
reg.score(X,y)
A = reg.coef_
b = reg.intercept_

In [158]:
reg.score(X, y)

0.6961376735166496